In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.integrate import solve_bvp


In [ ]:
track_points = np.column_stack((x_smooth / 20, y_smooth / 20))

seg_start = track_points[:-1]
seg_end   = track_points[1:]
seg_vec   = seg_end - seg_start
seg_len_sq = np.sum(seg_vec**2, axis=1)

def track_cost(x, y):
    Q = np.array([x, y])
    
    # Vector from start of each segment to Q
    w = Q - seg_start

    # Project w onto each segment vector
    # t_i = (w . seg_vec) / (seg_vec . seg_vec)
    t = np.sum(w * seg_vec, axis=1) / seg_len_sq

    # Clamp t to [0, 1] so projection stays within each segment
    t = np.clip(t, 0.0, 1.0)

    # Compute the projection point on each segment
    proj = seg_start + (t[:, np.newaxis] * seg_vec)

    # Compute squared distances from Q to each projection
    dists_sq = np.sum((Q - proj)**2, axis=1)

    # The distance to the track is the minimum distance to any segment
    min_dist = np.sqrt(np.min(dists_sq))

    cost = min_dist**10

    return cost